In [3]:
# Ensure src/ is in sys.path so 'tools' can be imported
# %%
import sys, os
try:
    # ✅ Running from a Python script (.py file)
    TOOLS_PATH = os.path.abspath(os.path.join(os.path.dirname(__file__),"..",".."))
except NameError:
    # ✅ Running from a Jupyter notebook (__file__ is not defined)
    TOOLS_PATH = os.path.abspath(os.path.join(os.getcwd(), "..",".."))
SRC_PATH = os.path.join(TOOLS_PATH)


if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)
    print(f"✅ SRC path added: {SRC_PATH}")
else:
    print(f"🔁 SRC path already in sys.path: {SRC_PATH}")

✅ SRC path added: /home/prashant-agrawal/projects/company_talk2data/src


In [ ]:
# src/utils/audio/stt_assembly.py

import requests, time
from typing import Optional

class AssemblyAITranscriber:
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.headers = {
            "authorization": api_key,
            "content-type": "application/json"
        }

    def upload_audio(self, filepath: str) -> str:
        with open(filepath, 'rb') as f:
            response = requests.post(
                "https://api.assemblyai.com/v2/upload",
                headers={"authorization": self.api_key},
                data=f
            )
        return response.json()['upload_url']

    def transcribe(self, audio_url: str) -> Optional[str]:
        transcript_request = {
            "audio_url": audio_url,
            "language_code": "en-IN"
        }
        response = requests.post(
            "https://api.assemblyai.com/v2/transcript",
            json=transcript_request,
            headers=self.headers
        )
        transcript_id = response.json()['id']

        while True:
            poll = requests.get(
                f"https://api.assemblyai.com/v2/transcript/{transcript_id}",
                headers=self.headers
            )
            status = poll.json()['status']
            if status == 'completed':
                return poll.json()['text']
            elif status == 'error':
                return None
            time.sleep(2)
